In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pyvis import network as net
import random
%matplotlib inline

In [106]:
data = pd.read_csv("eu-car-trade.csv")
data["reporter"] = data["reporter"].astype(str)
data["partner"] = data["partner"].astype(str)
data.dtypes

DATAFLOW        object
LAST UPDATE     object
freq            object
reporter        object
partner         object
product          int64
flow             int64
indicators      object
TIME_PERIOD      int64
OBS_VALUE        int64
OBS_FLAG       float64
dtype: object

In [105]:
data = data[data["TIME_PERIOD"] == 2022] # Set year that we consider

,DATAFLOW,LAST UPDATE,freq,reporter,partner,product,flow,indicators,TIME_PERIOD,OBS_VALUE,OBS_FLAG


In [104]:
filteredData = data[data["OBS_VALUE"] > 200000000] # Only consider trade values over 200 million Euros
#filteredData = filteredData[filteredData['flow'] == 1]
filteredData = filteredData.reset_index()
filteredData

,index,DATAFLOW,LAST UPDATE,freq,reporter,partner,product,flow,indicators,TIME_PERIOD,OBS_VALUE,OBS_FLAG


In [99]:
codes = pd.read_csv("country-codes.csv") # Get country codes
codes.loc[codes["Code"] == "RS", "Code"] = "XS" # Correct Serbia's country code

In [100]:
colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) # Generate random colours for each node, so the edges can be coloured too
             for i in range(len(filteredData))]

In [101]:
visnet = net.Network(notebook = True, cdn_resources = "in_line", directed = True) # Create network

for idx, row in filteredData.iterrows():
  
    reporterImage = "https://flagicons.lipis.dev/flags/4x3/" + row['reporter'].lower() + ".svg" # Country icon URLs
    partnerImage = "https://flagicons.lipis.dev/flags/4x3/" + row['partner'].lower() + ".svg"
    
    reporterDegree = filteredData[filteredData["reporter"] == row["reporter"]]["reporter"].count() # Country degrees, which determine the size of the node
    partnerDegree = filteredData[filteredData["partner"] == row["partner"]]["partner"].count()
    
    visnet.add_node(n_id = row['reporter'], label = codes[codes["Code"] == row['reporter']]["Name"].item(), title = row['reporter'], image = reporterImage, shape = 'circularImage', size = int(reporterDegree) * 2, color = colors[idx])
    visnet.add_node(n_id = row['partner'], label = codes[codes["Code"] == row['partner']]["Name"].item(), title = row['partner'], image = partnerImage, shape = 'circularImage', size = int(partnerDegree) * 2, color = colors[idx])
    visnet.add_edge(source = row['reporter'], to = row['partner'], value = int(row['OBS_VALUE']) / 200000000)

visnet.inherit_edge_colors(True)
visnet.repulsion(spring_length = 500, node_distance = 300) # Spread the nodes out so that the graph is easier to process
visnet.show_buttons(filter_ = ["physics"])
visnet.show("network-2018.html")

In [55]:
visnet2 = net.Network(notebook = True, cdn_resources = "in_line", directed = True)
visnet2.add_node('node1', 'node 1', color = "red")
visnet2.add_node('node1', 'node 1', color = "blue")
visnet2.add_node('node2', 'node 1', color = "blue")
visnet2.show('test.html')